# TransferLearning

+ 参考kernel:[Transfer Learning with VGG-16 ConvNet LB 0.1850](https://www.kaggle.com/devm2024/transfer-learning-with-vgg-16-convnet-lb-0-1850)
+ 参考kernel:[Transfer Learning with VGG-16 CNN+AUG LB 0.1712](https://www.kaggle.com/devm2024/transfer-learning-with-vgg-16-cnn-aug-lb-0-1712)
+ 参考文章：[VERY DEEP CONVOLUTIONAL NETWORKS FOR LARGE-SCALE IMAGE RECOGNITION](https://arxiv.org/pdf/1409.1556.pdf)

改进要点：
1. 改进第三通道？？
2. 模型优化？？
3. 多结果集成？
4. 第二次训练 显存占用率降不下来，提示Memory error

In [1]:
#Mandatory imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
from scipy.ndimage.filters import uniform_filter
plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

In [2]:
train = pd.read_json("../ShipIceberg/Data/train.json")
target_train=train['is_iceberg']

# 一直cannot resolve memory block
# test = pd.read_json("../ShipIceberg/Data/test.json") 

In [3]:
import json
with open('../ShipIceberg/Data/test.json', 'r') as f:
    test = json.load(f)
    test=pd.DataFrame(test)
    
print('done!')

done!


In [4]:
target_train=train['is_iceberg']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
train['inc_angle']=pd.to_numeric(train['inc_angle'], errors='coerce')#We have only 133 NAs.
train['inc_angle']=train['inc_angle'].fillna(method='pad')
X_angle=train['inc_angle']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
X_test_angle=test['inc_angle']

In [5]:
# Isolation function.
def iso(arr):
    p = np.reshape(np.array(arr), [75,75]) >(np.mean(np.array(arr))+2*np.std(np.array(arr)))
    return p * np.reshape(np.array(arr), [75,75])

# Size in number of pixels of every isolated object.
def size(arr):     
    return np.sum(arr<-5)/(75*75)

# Feature engineering iso1 and iso2.
train['iso1'] = train.iloc[:, 0].apply(iso)
train['iso2'] = train.iloc[:, 1].apply(iso)

# train.head(5)


# Feature engineering s1 s2 and size.
train['s1'] = train.iloc[:,5].apply(size)
train['s2'] = train.iloc[:,6].apply(size)
train['size'] = train.s1+train.s2

X_size = train['size']

# Feature engineering iso1 and iso2.
test['iso1'] = test.iloc[:, 0].apply(iso)
test['iso2'] = test.iloc[:, 1].apply(iso)

# test.head(5)

test['s1'] = test.iloc[:,4].apply(size)
test['s2'] = test.iloc[:,5].apply(size)
test['size'] = test.s1+test.s2


X_test_size = test['size']


In [6]:
#Generate the training data
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])

'''
# 需要循环？？？
# let's see if despeckling has any influence on the images
# plot nine different instances with different windows and noise levels (band 1)
windows = [2, 4, 8] # can be tuple too if not symetric
noise_var = np.array([1, 2, 4])

# 0-2
Wi = 2
Nj = 2

for i in range(0,1604):
    band_1_linear = decibel_to_linear(X_band_1[i])
    band_2_linear = decibel_to_linear(X_band_2[i])

    noise_var_1 = np.round(np.var(band_1_linear)*noise_var,10)
    noise_var_2 = np.round(np.var(band_2_linear)*noise_var,10)

    X_band_1[i] = linear_to_decibel(lee_filter(band_1_linear, windows[Wi], noise_var_1[Nj]))
    X_band_2[i] = linear_to_decibel(lee_filter(band_2_linear, windows[Wi], noise_var_2[Nj]))


# Sobel Operator & Averaging
from scipy import signal

Gx_sobel=np.array([[1,0,-1],[2,0,-2],[1,0,-1]])
Gy_sobel=np.array([[1,2,1],[0,0,0],[-1,-2,-1]])
lpf=1.0/25*np.ones((5,5))

for i in range(0,1604):
    
    img_1=X_band_1[i].copy()
    img_lpf_1=signal.convolve2d(img_1,lpf,mode='same')
    img_Gx_1=signal.convolve2d(img_lpf_1,Gx_sobel,mode='same')
    img_Gy_1=signal.convolve2d(img_lpf_1,Gy_sobel,mode='same')

    img_2=X_band_2[i].copy()
    img_lpf_2=signal.convolve2d(img_2,lpf,mode='same')
    img_Gx_2=signal.convolve2d(img_lpf_2,Gx_sobel,mode='same')
    img_Gy_2=signal.convolve2d(img_lpf_2,Gy_sobel,mode='same')

    X_band_1[i] = np.hypot(img_Gx_1,img_Gy_1)
    X_band_2[i] = np.hypot(img_Gx_2,img_Gy_2)
'''

X_band_3 = np.zeros((1604,75,75))
for i in range(0,1604):
    subt = abs(X_band_1[i]-X_band_2[i])
    W1 = subt/subt.max()
    W2=1-W1
    X_band_3[i]=W1 * X_band_1[i]+W2 * X_band_2[i]


# X_band_3=(X_band_1+X_band_2)/2  # 算术平均：尝试其他比例？
#X_band_3=X_band_1 * 0.5858 + X_band_2 * 0.4142
# 效果极差 X_band_3 = X_band_2 * X_band_2 / X_band_1
#X_band_3 = np.sqrt(X_band_1 * X_band_2)  # 调和·平均: sqrt invalid number???
#X_band_3 = X_band_1 / X_band_2
# X_band_3=X_band_2 * 2 - X_band_1

#from scipy import signal
#X_band_3=signal.fftconvolve(X_band_1, X_band_2, mode = 'same')

#X_band_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in train["inc_angle"]])

X_train = np.concatenate([X_band_1[:, :, :, np.newaxis]
                          , X_band_2[:, :, :, np.newaxis]
                         , X_band_3[:, :, :, np.newaxis]], axis=-1)


#Generate the test data
X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])

'''
for i in range(0,8424):
    band_test_1_linear = decibel_to_linear(X_band_test_1[i])
    band_test_2_linear = decibel_to_linear(X_band_test_2[i])

    noise_var_test_1 = np.round(np.var(band_test_1_linear)*noise_var,10)
    noise_var_test_2 = np.round(np.var(band_test_2_linear)*noise_var,10)

    X_band_test_1[i] = linear_to_decibel(lee_filter(band_test_1_linear, windows[Wi], noise_var_test_1[Nj]))
    X_band_test_2[i] = linear_to_decibel(lee_filter(band_test_2_linear, windows[Wi], noise_var_test_2[Nj]))


for i in range(0,8424):
    
    img_1=X_band_test_1[i].copy()
    img_lpf_1=signal.convolve2d(img_1,lpf,mode='same')
    img_Gx_1=signal.convolve2d(img_lpf_1,Gx_sobel,mode='same')
    img_Gy_1=signal.convolve2d(img_lpf_1,Gy_sobel,mode='same')

    img_2=X_band_test_2[i].copy()
    img_lpf_2=signal.convolve2d(img_2,lpf,mode='same')
    img_Gx_2=signal.convolve2d(img_lpf_2,Gx_sobel,mode='same')
    img_Gy_2=signal.convolve2d(img_lpf_2,Gy_sobel,mode='same')

    X_band_test_1[i] = np.hypot(img_Gx_1,img_Gy_1)
    X_band_test_2[i] = np.hypot(img_Gx_2,img_Gy_2)
'''

X_band_test_3 = np.zeros((8424,75,75))
for i in range(0,8424):
    subt = abs(X_band_test_1[i]-X_band_test_2[i])
    W1 = subt/subt.max()
    W2=1-W1
    X_band_test_3[i]=W1 * X_band_test_1[i]+W2 * X_band_test_2[i]

# X_band_test_3=(X_band_test_1+X_band_test_2)/2
#X_band_test_3=X_band_test_1 * 0.5858 + X_band_test_2 * 0.4142
#X_band_test_3 = X_band_test_2 * X_band_test_2 / X_band_test_1
#X_band_test_3 = np.sqrt(X_band_test_1*X_band_test_2)
#X_band_test_3=signal.fftconvolve(X_band_test_1, X_band_test_2, mode = 'same')
#X_band_test_3 = X_band_test_1 / X_band_test_2
# X_band_test_3=X_band_test_2 * 2 - X_band_test_1

#X_band_test_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in test["inc_angle"]])

X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis]
                          , X_band_test_2[:, :, :, np.newaxis]
                         , X_band_test_3[:, :, :, np.newaxis]], axis=-1)

In [7]:
#Import Keras.
from matplotlib import pyplot
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers

from keras.optimizers import Adam
#from keras.optimizers import rmsprop
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from keras.optimizers import Adagrad
from keras.optimizers import Adadelta
from keras.optimizers import Adamax
from keras.optimizers import Nadam


from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.datasets import cifar10
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import VGG19
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
#from keras.applications.vgg19 import preprocess_input
#from keras.applications.xception import preprocess_input
#from keras.applications.inception_v3 import preprocess_input
# ResNet

#Data Aug for multi-input
from keras.preprocessing.image import ImageDataGenerator

# 批处理大小的设置
# batch_size=64
batch_size=64

# Define the image transformations here
# 原始旋转范围10
#gen = ImageDataGenerator(horizontal_flip = True,
#                         vertical_flip = True,
#                         width_shift_range = 0.,
#                         height_shift_range = 0.,
#                         channel_shift_range=0,
#                         zoom_range = 0.2,
#                         rotation_range = 10)



# aug1
gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.,
                         height_shift_range = 0.,
                         channel_shift_range=0.,
                         zoom_range = 0.,
                         rotation_range = 0)


# Here is the function that merges our two generators
# We use the exact same generator with the same random seed for both the y and angle arrays
def gen_flow_for_two_inputs(X1, X2, X3, y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=55)
    genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=55)
    genX3 = gen.flow(X1,X3, batch_size=batch_size,seed=55)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            X3i = genX3.next()
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], X2i[1], X3i[1]], X1i[1]

# Finally create generator
def get_callbacks(filepath, patience=2):
   es = EarlyStopping('val_loss', patience=10, mode="min")
   msave = ModelCheckpoint(filepath, save_best_only=True)
   return [es, msave]


def getVggAngleModel():
    input_2 = Input(shape=[1], name="angle")
    angle_layer = Dense(1, )(input_2)
    
    input_3 = Input(shape=[1], name="size")
    size_layer = Dense(1, )(input_3)
    
    # VGG16换成其他模型？？
    base_model = VGG16(weights='imagenet', include_top=False, 
                 input_shape=X_train.shape[1:], classes=1)
    x = base_model.get_layer('block5_pool').output
  
    '''
    # BatchNorm 未形成conv bn scale relu 的block
    input_tensor = Input(shape=X_train.shape[1:])
    bn = BatchNormalization()(input_tensor)
#     x = base_model(bn)
    x = base_model.get_layer('block5_pool')(bn)
'''
#     x = GlobalMaxPooling2D()(x)
    x = Flatten()(x)
    merge_one = concatenate([x, angle_layer,size_layer])
#     merge_one = x
    merge_one = Dense(512, activation='relu', name='fc2')(merge_one)
#     merge_one = Dense(1024, activation='relu', name='fc2')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    merge_one = Dense(512, activation='relu', name='fc3')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    
    predictions = Dense(1, activation='sigmoid')(merge_one)
    
    model = Model(input=[base_model.input, input_2,input_3], output=predictions)
#     model = Model(input=[base_model.input], output=predictions)
#     model = Model(input=[input_tensor, input_2], output=predictions)
    
    # 使用不同的优化
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    adagrad = Adagrad(lr = 1e-3, epsilon = 1e-6)
    rmsprop = RMSprop(lr=1e-3, rho = 0.9, epsilon=1e-6)
    adadelta = Adadelta(lr=1e-3, rho=0.95, epsilon=1e-06)
    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    adamax = Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    nadam = Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
    
    # 更换loss
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    
    return model

#Using K-fold Cross Validation with Data Augmentation.
def myAngleCV(X_train, X_angle, X_size, X_test):
    # K-折交叉验证
    K=4
    
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=16).split(X_train, target_train))
    y_test_pred_log = 0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    
    for j, (train_idx, test_idx) in enumerate(folds):
        print('\n===================FOLD=',j)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_holdout = X_train[test_idx]
        Y_holdout= target_train[test_idx]
        
        #Angle
        X_angle_cv=X_angle[train_idx]
        X_angle_hold=X_angle[test_idx]
        
        X_size_cv=X_size[train_idx]
        X_size_hold=X_size[test_idx]

        #define file path and get callbacks
        file_path = "./model/%s_aug_model_weights.hdf5"%j
        callbacks = get_callbacks(filepath=file_path, patience=5)
        gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv,X_size_cv, y_train_cv)
        galaxyModel= getVggAngleModel()
        
        # 调整训练参数
        galaxyModel.fit_generator(
                gen_flow,
#                 steps_per_epoch=24,
                steps_per_epoch=len(X_train_cv)//batch_size,
                #steps_per_epoch=100,
                epochs=100,
                shuffle=True,
                verbose=1,
                validation_data=([X_holdout,X_angle_hold,X_size_hold], Y_holdout),
                callbacks=callbacks)

        #Getting the Best Model
        galaxyModel.load_weights(filepath=file_path)
        #Getting Training Score
        score = galaxyModel.evaluate([X_train_cv,X_angle_cv, X_size_cv], y_train_cv, verbose=0)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = galaxyModel.evaluate([X_holdout,X_angle_hold, X_size_hold], Y_holdout, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        #Getting validation Score.
        pred_valid=galaxyModel.predict([X_holdout,X_angle_hold, X_size_hold])
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
        temp_test=galaxyModel.predict([X_test, X_test_angle, X_test_size])
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=galaxyModel.predict([X_train, X_angle, X_size])
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_test_pred_log=y_test_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log

Using TensorFlow backend.


In [8]:
preds=myAngleCV(X_train, X_angle, X_size, X_test)


===================FOLD= 0


/home/jayden/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:120: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


Epoch 1/100
18/18 [==============================] - 19s - loss: 0.6591 - acc: 0.6033 - val_loss: 0.4187 - val_acc: 0.7935
Epoch 2/100
18/18 [==============================] - 10s - loss: 0.5005 - acc: 0.7373 - val_loss: 0.4131 - val_acc: 0.7587
Epoch 3/100
18/18 [==============================] - 8s - loss: 0.3482 - acc: 0.8242 - val_loss: 0.4746 - val_acc: 0.7761
Epoch 4/100
18/18 [==============================] - 9s - loss: 0.3219 - acc: 0.8681 - val_loss: 0.2475 - val_acc: 0.8881
Epoch 5/100
18/18 [==============================] - 9s - loss: 0.2611 - acc: 0.8855 - val_loss: 0.2338 - val_acc: 0.8806
Epoch 6/100
18/18 [==============================] - 9s - loss: 0.2131 - acc: 0.9120 - val_loss: 0.2224 - val_acc: 0.9030
Epoch 7/100
18/18 [==============================] - 9s - loss: 0.2443 - acc: 0.9057 - val_loss: 0.2106 - val_acc: 0.9030
Epoch 8/100
18/18 [==============================] - 8s - loss: 0.2224 - acc: 0.9092 - val_loss: 0.2066 - val_acc: 0.9080
Epoch 9/100
18/18 [===

18/18 [==============================] - 8s - loss: 0.1573 - acc: 0.9308 - val_loss: 0.2604 - val_acc: 0.8953
Epoch 19/100
18/18 [==============================] - 8s - loss: 0.1349 - acc: 0.9477 - val_loss: 0.2489 - val_acc: 0.8953
Epoch 20/100
18/18 [==============================] - 8s - loss: 0.1416 - acc: 0.9418 - val_loss: 0.2541 - val_acc: 0.9002
Epoch 21/100
18/18 [==============================] - 8s - loss: 0.1017 - acc: 0.9627 - val_loss: 0.3018 - val_acc: 0.9052
Epoch 22/100
18/18 [==============================] - 8s - loss: 0.1194 - acc: 0.9497 - val_loss: 0.2669 - val_acc: 0.9152
Epoch 23/100
18/18 [==============================] - 8s - loss: 0.1146 - acc: 0.9581 - val_loss: 0.2869 - val_acc: 0.9102
Epoch 24/100
18/18 [==============================] - 8s - loss: 0.1147 - acc: 0.9538 - val_loss: 0.2526 - val_acc: 0.9127
Epoch 25/100
18/18 [==============================] - 8s - loss: 0.1111 - acc: 0.9535 - val_loss: 0.2881 - val_acc: 0.9077
Epoch 26/100
18/18 [=========

In [9]:
#Submission for each day.
submission = pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=preds
submission.to_csv('./submission/subVGG7.18.csv', index=False)